In [1]:
# 필요한 라이브러리 import
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os

# OpenAI API 키 설정 (환경변수에서 가져오기)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"


In [2]:
# FewShot 예시 정의
examples = [
    {
        "input": "탑건",
        "output": "🛩️👨‍✈️🔥"
    },
    {
        "input": "대부", 
        "output": "👨‍👨‍👦🔫🍝"
    },
    {
        "input": "어벤져스",
        "output": "🦸‍♂️💥⚡"
    },
    {
        "input": "타이타닉",
        "output": "🚢💔🌊"
    }
]

# 프롬프트 템플릿 정의
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="영화: {input}\n이모티콘: {output}"
)

# FewShot 프롬프트 템플릿 생성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 영화 제목을 세 개의 이모티콘으로 표현해주세요. 항상 정확히 세 개의 이모티콘만 사용하세요:",
    suffix="영화: {input}\n이모티콘:",
    input_variables=["input"]
)


In [3]:
# LLM과 메모리 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory(return_messages=True)

# LCEL 체인 구성
def create_movie_emoji_chain():
    # 프롬프트를 문자열로 변환
    prompt_template = few_shot_prompt.format(input="{input}")
    
    # ChatPromptTemplate으로 변환
    chat_prompt = ChatPromptTemplate.from_template(prompt_template)
    
    # LCEL 체인 구성
    chain = (
        {"input": RunnablePassthrough()}
        | chat_prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

# 체인 생성
movie_emoji_chain = create_movie_emoji_chain()

print("영화 이모티콘 변환 체인이 준비되었습니다!")


영화 이모티콘 변환 체인이 준비되었습니다!


/tmp/ipykernel_7775/2552589832.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [4]:
# 첫 번째 영화 테스트
movie1 = "인터스텔라"
result1 = movie_emoji_chain.invoke({"input": movie1})
print(f"영화: {movie1}")
print(f"이모티콘: {result1}")

# 메모리에 대화 저장
memory.save_context({"input": f"영화: {movie1}"}, {"output": f"이모티콘: {result1}"})

print("\n" + "="*50 + "\n")

# 두 번째 영화 테스트  
movie2 = "해리포터"
result2 = movie_emoji_chain.invoke({"input": movie2})
print(f"영화: {movie2}")
print(f"이모티콘: {result2}")

# 메모리에 대화 저장
memory.save_context({"input": f"영화: {movie2}"}, {"output": f"이모티콘: {result2}"})


영화: 인터스텔라
이모티콘: 🚀🌌⏳


영화: 해리포터
이모티콘: ⚡🧙‍♂️🔮


In [5]:
# 메모리 기능 테스트 - 이전에 질문한 영화들 확인
print("=== 메모리 테스트 ===")
print("이전에 질문한 영화들을 확인해보겠습니다:\n")

# 메모리 내용 출력
memory_variables = memory.load_memory_variables({})
print("저장된 대화 내용:")
for i, message in enumerate(memory_variables['history']):
    print(f"{i+1}. {message.content}")

print("\n" + "="*50 + "\n")

# 메모리를 사용한 질문 체인 생성
def create_memory_chain():
    from langchain_core.prompts import ChatPromptTemplate
    
    # 메모리와 함께 사용할 프롬프트
    memory_prompt = ChatPromptTemplate.from_template("""
    다음은 이전 대화 내용입니다:
    {history}
    
    질문: {question}
    답변:""")
    
    # 메모리를 포함한 체인
    chain = (
        {"history": lambda x: memory.load_memory_variables({})['history'], 
         "question": RunnablePassthrough()}
        | memory_prompt
        | llm
        | StrOutputParser()
    )
    
    return chain

# 메모리 체인 생성 및 테스트
memory_chain = create_memory_chain()

# 이전에 질문한 영화에 대해 물어보기
question = "제가 처음에 질문한 영화는 무엇인가요?"
answer = memory_chain.invoke(question)
print(f"질문: {question}")
print(f"답변: {answer}")


=== 메모리 테스트 ===
이전에 질문한 영화들을 확인해보겠습니다:

저장된 대화 내용:
1. 영화: 인터스텔라
2. 이모티콘: 🚀🌌⏳
3. 영화: 해리포터
4. 이모티콘: ⚡🧙‍♂️🔮


질문: 제가 처음에 질문한 영화는 무엇인가요?
답변: 인터스텔라입니다.
